Import the preprocessed LIAR dataset

In [240]:
from datasets import load_dataset

dataset = load_dataset(
    "csv", 
    data_files={
        'train': "preprocessed_liar/liar_train.csv", 'test': "preprocessed_liar/liar_test.csv", 'valid': "preprocessed_liar/liar_valid.csv"})
print(dataset)

KeyboardInterrupt: 

Pick the pre-trained model

In [ ]:
model_name = "albert/albert-base-v2"
your_path = 'liar_new_results'

Look over the label distribution

In [ ]:
from collections import Counter

train_label_distribution = Counter(dataset['train']['label'])
test_label_distribution = Counter(dataset['test']['label'])

print("Training Label Distribution:", train_label_distribution)
print("Test Label Distribution:", test_label_distribution)

Labels in their original form are strings (true/false). We need to convert them to numerical values so they can be processed by the model.

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

label_encoder.fit(dataset['train']['label'])

def encode_labels(example):
    return {'encoded_label': label_encoder.transform([example['label']])[0]}

for split in dataset:
    dataset[split] = dataset[split].map(encode_labels, batched=False)

The id2label and label2id mappings in AutoConfig are used to inform the model of the specific label-to-ID mappings so we can get the actual label names rather than the numerical reps when we do inference with the model.

In [ ]:
from transformers import AutoConfig

unique_labels = sorted(list(set(dataset['train']['label'])))
id2label = {i: label for i, label in enumerate(unique_labels)}
label2id = {label: i for i, label in enumerate(unique_labels)}

config = AutoConfig.from_pretrained(model_name)
config.id2label = id2label
config.label2id = label2id

Verify the correct labels are being used

In [ ]:
print("ID to Label Mapping:", config.id2label)
print("Label to ID Mapping:", config.label2id)

Now, we need to tokenize the text data so it can be processed by the model. We will use the tokenizer that corresponds to the model we are using from Hugging Face's Transformers library.

In [ ]:
from transformers import AlbertForSequenceClassification, AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertForSequenceClassification.from_pretrained(model_name, config=config)

In [ ]:
print(model)

We need to filter out any examples that have invalid content (e.g. empty strings) before tokenizing the data. Then, we will encode the labels and tokenize the text data. Tokenizing means converting the text data into numerical representations that can be processed by the model.

In [ ]:
def filter_invalid_content(example):
    return isinstance(example['statement'], str)

dataset = dataset.filter(filter_invalid_content, batched=False)

def encode_data(batch):
    tokenized_inputs = tokenizer(batch["statement"], padding=True, truncation=True, max_length=256)
    tokenized_inputs["labels"] = batch["encoded_label"]
    return tokenized_inputs

Verify the data is tokenized correctly:

In [ ]:
dataset_encoded = dataset.map(encode_data, batched=True)
print(dataset_encoded) 

The DataCollatorWithPadding ensures that all input sequences in a batch are padded to the same length, using the padding logic defined by the tokenizer. This is necessary because the model can only process inputs of the same length.

In [ ]:
dataset_encoded.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

Next we'll set up LabelEncoder to encode labels and defines a function to compute per-label accuracy from a confusion matrix, providing label-specific accuracy metrics. I.e. when we train the model we want to see the accuracy metrics per label as well as the average metrics. This is more relevant if you have more than two labels, and one is underperforming. 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import numpy as np

label_encoder = LabelEncoder()
label_encoder.fit(unique_labels)

def per_label_accuracy(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    correct_predictions = cm.diagonal()
    label_totals = cm.sum(axis=1)
    per_label_acc = np.divide(correct_predictions, label_totals, out=np.zeros_like(correct_predictions, dtype=float), where=label_totals != 0)
    return dict(zip(labels, per_label_acc))

Compute the following metrics: accuracy, precision, recall, f1 score, and per-label accuracy.

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    decoded_labels = label_encoder.inverse_transform(labels)
    decoded_preds = label_encoder.inverse_transform(preds)

    precision = precision_score(decoded_labels, decoded_preds, average='weighted')
    recall = recall_score(decoded_labels, decoded_preds, average='weighted')
    f1 = f1_score(decoded_labels, decoded_preds, average='weighted')
    acc = accuracy_score(decoded_labels, decoded_preds)

    labels_list = list(label_encoder.classes_)
    per_label_acc = per_label_accuracy(decoded_labels, decoded_preds, labels_list)

    per_label_acc_metrics = {}
    for label, accuracy in per_label_acc.items():
        label_key = f"accuracy_label_{label}"
        per_label_acc_metrics[label_key] = accuracy
        
    mean =  np.mean(list(per_label_acc_metrics.values()))
    std = np.std(list(per_label_acc_metrics.values()))
    per_label_acc_metrics['mean'] = mean
    per_label_acc_metrics['std'] = std

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        **per_label_acc_metrics
    }

Next, the training begins. Training loss and validation loss should decrease consistenly. If the training loss is decreasing but the validation loss is increasing, the model is overfitting. If both are increasing, the model is underfitting.

In [ ]:
import time
from sklearn.model_selection import ParameterGrid
from transformers import Trainer, TrainingArguments

param_grid = {
    'num_train_epochs': [3, 4, 5],
    'learning_rate': [2e-5, 3e-5, 4e-5],
    'weight_decay': [0.01, 0.1],
    'per_device_train_batch_size': [16, 32], 
}


best_score = -np.inf
best_params = {}
execution_times = []  

for params in ParameterGrid(param_grid):
    print(f"Training with parameters: {params}")

    training_args = TrainingArguments(
        output_dir='./output',
        num_train_epochs=params['num_train_epochs'],
        per_device_train_batch_size=params['per_device_train_batch_size'],
        per_device_eval_batch_size=16,
        weight_decay=params['weight_decay'],
        logging_steps=10,
        eval_strategy='steps',
        eval_steps=100,
        save_strategy='epoch',
        learning_rate=params['learning_rate'],
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_encoded['train'],
        eval_dataset=dataset_encoded['test'],
        compute_metrics=compute_metrics,
        data_collator=data_collator,
    )
    start_time = time.time()

    trainer.train()

    end_time = time.time()
    training_time = end_time - start_time
    execution_times.append(training_time)
    print(f"Training time: {training_time:.2f} seconds")

    results = trainer.evaluate(dataset_encoded['test'])
    score = results['eval_accuracy']  

    print(f"Validation accuracy: {score}")

    if score > best_score:
        best_score = score
        best_params = params

print(f"Best parameters: {best_params}")
print(f"Execution times for each configuration: {execution_times}")


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.1,
    logging_steps=50,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='epoch',
    learning_rate=4e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_encoded['train'],
    eval_dataset=dataset_encoded['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Save the results, the model and the state of the model.

In [ ]:
result = trainer.evaluate(dataset_encoded['valid'])
print(result)
trainer.save_model(your_path)
trainer.save_state()

In [ ]:
print(result)

In [ ]:
import random
from sklearn.model_selection import StratifiedKFold
from transformers import Trainer, TrainingArguments

labels = dataset_encoded['train']['label']

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, test_idx) in enumerate(stratified_kfold.split(dataset_encoded['train'], labels)):
    print(f"Fold: {fold}")
    train_dataset = dataset_encoded['train'].select(train_idx)
    test_dataset = dataset_encoded['train'].select(test_idx)

    seed = random.randint(0, 10000)
    np.random.seed(seed)
    random.seed(seed)

    training_args = TrainingArguments(
        output_dir=f'./output_fold_{fold}',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.1,
        logging_steps=50,
        eval_strategy='steps',
        eval_steps=100,
        save_strategy='epoch',
        learning_rate=4e-5,
        seed=seed
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
        data_collator=data_collator,
    )

    trainer.train()
    result = trainer.evaluate(dataset_encoded['valid'])
    print(result)
